필요한 라이브러리 불러오기


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

from xgboost.sklearn import XGBClassifier,XGBRegressor
from lightgbm.sklearn import LGBMClassifier,LGBMRegressor

from sklearn.model_selection import  KFold
from functools import partial

import optuna


데이터 불러오기

In [2]:
# data 폴더에 있는 모든 csv 파일을 읽어오기 위해 glob을 사용합니다.
from glob import glob

file_paths = sorted(sorted(glob('C:/Users/jeong bok/Documents/cse101/team9PJ/mlops-team9/data/*.xlsx')))
# csv 목록 불러오기
total = pd.DataFrame()
for file_path in file_paths:
    temp = pd.read_excel(file_path,sheet_name=1)
    temp.fillna('0',inplace=True)
    total = pd.concat([total,temp])


# 모든 csv 병합하기
total

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
0,20181001,월,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,419.0,308.0,227.0,191.0,...,1925.0,2012.0,2104.0,2301.0,2246.0,1743.0,1400.0,1508.0,1331.0,983.0
1,20181002,화,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,669.0,508.0,324.0,283.0,...,1977.0,2128.0,2221.0,2394.0,2306.0,1961.0,1685.0,1597.0,1536.0,1013.0
2,20181003,수,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,785.0,547.0,479.0,396.0,...,1463.0,1602.0,1719.0,1768.0,1640.0,1405.0,1292.0,1247.0,1007.0,721.0
3,20181004,목,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,507.0,323.0,198.0,231.0,...,1970.0,2050.0,2311.0,2451.0,2198.0,1796.0,1527.0,1557.0,1457.0,991.0
4,20181005,금,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,701.0,595.0,389.0,337.0,...,2076.0,2029.0,2070.0,2137.0,2157.0,1998.0,1406.0,1392.0,1236.0,977.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8095,20180926,수,서부간선도로,F-09,유출,목동교->신정교,2338.0,1790.0,1412.0,1147.0,...,2137.0,3135.0,3001.0,3016.0,2875.0,3085.0,3158.0,3168.0,3062.0,2515.0
8096,20180927,목,서부간선도로,F-09,유출,목동교->신정교,1893.0,1269.0,910.0,842.0,...,3122.0,3086.0,2925.0,2901.0,2797.0,3014.0,3214.0,3414.0,3280.0,3075.0
8097,20180928,금,서부간선도로,F-09,유출,목동교->신정교,2414.0,1715.0,1253.0,959.0,...,3123.0,2994.0,2894.0,3004.0,2684.0,2827.0,3092.0,3249.0,3123.0,3316.0
8098,20180929,토,서부간선도로,F-09,유출,목동교->신정교,3014.0,2640.0,2066.0,1785.0,...,2962.0,2971.0,2855.0,2868.0,2829.0,2865.0,2965.0,3122.0,3006.0,3131.0


In [3]:
total.head(5)
total.tail(5)

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
8095,20180926,수,서부간선도로,F-09,유출,목동교->신정교,2338.0,1790.0,1412.0,1147.0,...,2137.0,3135.0,3001.0,3016.0,2875.0,3085.0,3158.0,3168.0,3062.0,2515.0
8096,20180927,목,서부간선도로,F-09,유출,목동교->신정교,1893.0,1269.0,910.0,842.0,...,3122.0,3086.0,2925.0,2901.0,2797.0,3014.0,3214.0,3414.0,3280.0,3075.0
8097,20180928,금,서부간선도로,F-09,유출,목동교->신정교,2414.0,1715.0,1253.0,959.0,...,3123.0,2994.0,2894.0,3004.0,2684.0,2827.0,3092.0,3249.0,3123.0,3316.0
8098,20180929,토,서부간선도로,F-09,유출,목동교->신정교,3014.0,2640.0,2066.0,1785.0,...,2962.0,2971.0,2855.0,2868.0,2829.0,2865.0,2965.0,3122.0,3006.0,3131.0
8099,20180930,일,서부간선도로,F-09,유출,목동교->신정교,2953.0,2258.0,1784.0,1389.0,...,2957.0,3122.0,3035.0,2994.0,2851.0,2861.0,3117.0,3117.0,2838.0,2528.0


In [10]:


tr_df = total
air_df = pd.read_csv('C:/Users/jeong bok/Documents/cse101/team9PJ/mlops-team9/data/new_AIR_HOUR_2018.csv')

gu_df = pd.read_csv('C:/Users/jeong bok/Documents/cse101/team9PJ/mlops-team9/data/spot_data_guz.csv')

In [11]:
tr_df['일자'] = pd.to_datetime(tr_df['일자'], format='%Y%m%d')

# '일자'를 기반으로 데이터프레임 정렬
tr_df = tr_df.sort_values(by='일자').reset_index(drop=True)

In [12]:
tr_df

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
0,2018-01-01,월,영등포로(오목교),D-25,유출,관악고교->오목교동측,557.0,754.0,421.0,306.0,...,1477.0,1517.0,1636.0,1589.0,1535.0,1397.0,1307.0,1202.0,1016.0,730.0
1,2018-01-01,월,광명대교(광명시계),B-22,유입,광명대교사거리->구로IC,362.0,421.0,291.0,200.0,...,972.0,967.0,992.0,895.0,696.0,706.0,581.0,521.0,451.0,334.0
2,2018-01-01,월,동작대교,C-10,유입,동작대교남단->동작대교북단,650.0,577.0,432.0,297.0,...,1530.0,1273.0,1405.0,1187.0,1022.0,960.0,927.0,807.0,648.0,454.0
3,2018-01-01,월,동작대교,C-10,유출,동작대교북단->동작대교남단,820.0,861.0,454.0,313.0,...,1475.0,1629.0,1802.0,1596.0,1292.0,1182.0,1185.0,1120.0,810.0,530.0
4,2018-01-01,월,강남대로(강남역-신분당),D-35,유출,강남역->뱅뱅사거리,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98545,2018-12-31,월,노들로,D-41,유출,여의2교->여의하류IC(노들로),485.0,370.0,304.0,206.0,...,2327.0,2395.0,2468.0,2017.0,1789.0,2133.0,1709.0,1534.0,1413.0,968.0
98546,2018-12-31,월,철산교(광명시계),B-23,유입,철산대교사거리->수출의다리,0,0,0,109.0,...,915.0,838.0,726.0,819.0,697.0,727.0,709.0,624.0,684.0,475.0
98547,2018-12-31,월,철산교(광명시계),B-23,유출,수출의다리->철산대교사거리,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98548,2018-12-31,월,금오로(광명시계),B-19,유입,천왕지구삼거리->연지마을,22.0,26.0,18.0,19.0,...,404.0,420.0,445.0,505.0,456.0,319.0,210.0,169.0,131.0,91.0


In [13]:
# 'time' 열의 값이 단일 자릿수인 경우, 두 자리로 맞추기
# air_df['time'] = air_df['time'].astype(int)
# air_df['time'].dtype
#df_air['datetime']=df_air[df_air['date']+df_air['time']]


In [14]:
tr_df.nunique()

일자       365
요일         7
지점명      190
지점번호     135
방향       324
구분       271
0시      4222
1시      3260
2시      2654
3시      2232
4시      2558
5시      4752
6시      6569
7시      6834
8시      6770
9시      6679
10시     6737
11시     6784
12시     6736
13시     6864
14시     6848
15시     6858
16시     6819
17시     6615
18시     6422
19시     6232
20시     6261
21시     6466
22시     6278
23시     5344
dtype: int64

In [15]:
air_df

,Unnamed: 0,날짜,시간,구,PM10,PM2.5,O3,SO2,NO2,CO
0,0,2018-01-01,0,종로구,29.0,17.0,0.022,0.006,0.021,0.4
1,1,2018-01-01,0,중구,34.0,19.0,0.020,0.004,0.020,0.5
2,2,2018-01-01,0,용산구,29.0,14.0,0.013,0.004,0.023,0.3
3,3,2018-01-01,0,은평구,43.0,25.0,0.017,0.003,0.018,0.6
4,4,2018-01-01,0,서대문구,0.0,0.0,0.000,0.000,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...
264746,264746,2018-12-31,23,관악구,36.0,24.0,0.004,0.005,0.059,0.6
264747,264747,2018-12-31,23,서초구,36.0,13.0,0.002,0.004,0.052,0.5
264748,264748,2018-12-31,23,강남구,35.0,26.0,0.002,0.005,0.044,0.7
264749,264749,2018-12-31,23,송파구,43.0,27.0,0.002,0.004,0.057,0.9


In [16]:
tr_df.isna().value_counts()

일자     요일     지점명    지점번호   방향     구분     0시     1시     2시     3시     4시     5시     6시     7시     8시     9시     10시    11시    12시    13시    14시    15시    16시    17시    18시    19시    20시    21시    22시    23시  
False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False    98550
Name: count, dtype: int64

In [17]:
air_df.head(5)

,Unnamed: 0,날짜,시간,구,PM10,PM2.5,O3,SO2,NO2,CO
0,0,2018-01-01,0,종로구,29.0,17.0,0.022,0.006,0.021,0.4
1,1,2018-01-01,0,중구,34.0,19.0,0.020,0.004,0.020,0.5
2,2,2018-01-01,0,용산구,29.0,14.0,0.013,0.004,0.023,0.3
3,3,2018-01-01,0,은평구,43.0,25.0,0.017,0.003,0.018,0.6
4,4,2018-01-01,0,서대문구,0.0,0.0,0.000,0.000,0.000,0.0


In [18]:
gu_df.head(5)

,spot_num,gu
0,A-01,서대문구
1,A-02,종로구
2,A-03,종로구
3,A-04,성북구
4,A-05,종로구


데이터 전처리를 진행합니다

In [19]:
#칼럼값으로 퍼져있는 시간을 로우로 내림
melted_df = pd.melt(tr_df, id_vars=['일자', '요일', '지점명', '지점번호', '구분', '방향'],
                    var_name='시간', value_name='값')

In [20]:
melted_df.head(5)

,일자,요일,지점명,지점번호,구분,방향,시간,값
0,2018-01-01,월,영등포로(오목교),D-25,관악고교->오목교동측,유출,0시,557.0
1,2018-01-01,월,광명대교(광명시계),B-22,광명대교사거리->구로IC,유입,0시,362.0
2,2018-01-01,월,동작대교,C-10,동작대교남단->동작대교북단,유입,0시,650.0
3,2018-01-01,월,동작대교,C-10,동작대교북단->동작대교남단,유출,0시,820.0
4,2018-01-01,월,강남대로(강남역-신분당),D-35,강남역->뱅뱅사거리,유출,0시,0


In [21]:
melted_df.isna()

,일자,요일,지점명,지점번호,구분,방향,시간,값
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
2365195,False,False,False,False,False,False,False,False
2365196,False,False,False,False,False,False,False,False
2365197,False,False,False,False,False,False,False,False
2365198,False,False,False,False,False,False,False,False


In [22]:
#결측치 확인 후 채움
melted_df.isna().value_counts()

melted_df.fillna('0',inplace=True)

In [23]:
melted_df.isna().sum()

일자      0
요일      0
지점명     0
지점번호    0
구분      0
방향      0
시간      0
값       0
dtype: int64

칼럼명 영어로 변경

In [24]:
new_column_names = {
    '일자': 'date',
    '시간': 'time',
    '지점번호': 'spot_num',
    '지점명': 'spot_nm',
    '요일' : 'day' ,
    '값' : 'traffic',
    '날짜' : 'date',
    '구'   : 'gu'
}

melted_df.rename(columns=new_column_names, inplace=True)
air_df.rename(columns=new_column_names, inplace=True)

In [25]:
# 필요없다고 생각된 구분 칼럼 삭제
melted_df = melted_df.drop(melted_df.columns[4], axis=1)

In [26]:

air_df = air_df.drop(air_df.columns[0], axis=1)

In [27]:
melted_df

,date,day,spot_nm,spot_num,방향,time,traffic
0,2018-01-01,월,영등포로(오목교),D-25,유출,0시,557.0
1,2018-01-01,월,광명대교(광명시계),B-22,유입,0시,362.0
2,2018-01-01,월,동작대교,C-10,유입,0시,650.0
3,2018-01-01,월,동작대교,C-10,유출,0시,820.0
4,2018-01-01,월,강남대로(강남역-신분당),D-35,유출,0시,0
...,...,...,...,...,...,...,...
2365195,2018-12-31,월,노들로,D-41,유출,23시,968.0
2365196,2018-12-31,월,철산교(광명시계),B-23,유입,23시,475.0
2365197,2018-12-31,월,철산교(광명시계),B-23,유출,23시,0
2365198,2018-12-31,월,금오로(광명시계),B-19,유입,23시,91.0


In [28]:
#데이트타임 적용
# melted_df['time']=melted_df['time'].str.replace('시','')
# melted_df['datetime'] = pd.to_datetime(melted_df['date'].astype(str) + ' ' + melted_df['time'], format='%Y%m%d %H')
# melted_df['datetime'] = melted_df['datetime'].dt.floor('H')
# melted_df.drop(['date', 'time'], axis=1, inplace=True)



In [29]:
traffic_df=melted_df

In [30]:

# # 'time' 열의 값이 단일 자릿수인 경우, 두 자리로 맞추기
# air_df['time'] = air_df['time'].astype(int)
# air_df['time'].dtype
# #df_air['datetime']=df_air[df_air['date']+df_air['time']]



# # 'time' 열을 문자열로 변환하고, 두 자리로 만듭니다.
# air_df['time'] = air_df['time'].astype(str).str.zfill(2)

# # 'datetime' 열 생성
# air_df['datetime'] = pd.to_datetime(air_df['date'].astype(str) + ' ' + air_df['time'], format='%Y-%m-%d %H', errors='coerce')

# # 결과 확인
# print(air_df.head())


# # 'time' 열을 문자열로 변환
# air_df['time'] = air_df['time'].astype(str)

# # 'time' 열의 고유값 확인
# print(air_df['time'].unique())
# dr_col = [
#     'date','time'
# ]
# air_df.drop(columns=dr_col)

In [35]:
merged_df = pd.merge(melted_df, gu_df, how='left', left_on='spot_num', right_on='spot_num')

In [36]:
drop_col = ['spot_num','day']
merged_df =merged_df.drop(columns=drop_col)

In [37]:
print(merged_df[['time', 'date', 'gu', 'traffic']].dtypes)
print(merged_df[['time', 'date', 'gu', 'traffic']].isnull().sum())


time               object
date       datetime64[ns]
gu                 object
traffic            object
dtype: object
time       0
date       0
gu         0
traffic    0
dtype: int64


In [38]:
merged_df['time'] = merged_df['time'].astype(str).str.replace('시', '').astype(int)
merged_df['traffic'] = merged_df['traffic'].astype(float)
# 방향을 기준으로 groupby하고, traffic을 합산
df_combined = merged_df.groupby(['time', 'date', 'gu'])['traffic'].sum().reset_index()

# # 방향 column은 필요 없으므로 제거
# df_combined = df_combined.drop(columns=['방향'])

# # 결과 확인
# print(df_combined)

In [39]:
df_combined.nunique()

time          24
date         365
gu            24
traffic    42873
dtype: int64

In [40]:
air_df.head(30)

,date,time,gu,PM10,PM2.5,O3,SO2,NO2,CO
0,2018-01-01,0,종로구,29.0,17.0,0.022,0.006,0.021,0.4
1,2018-01-01,0,중구,34.0,19.0,0.020,0.004,0.020,0.5
2,2018-01-01,0,용산구,29.0,14.0,0.013,0.004,0.023,0.3
3,2018-01-01,0,은평구,43.0,25.0,0.017,0.003,0.018,0.6
4,2018-01-01,0,서대문구,0.0,0.0,0.000,0.000,0.000,0.0
5,2018-01-01,0,마포구,34.0,19.0,0.014,0.005,0.026,0.5
6,2018-01-01,0,성동구,33.0,19.0,0.017,0.005,0.018,0.0
7,2018-01-01,0,성동구,0.0,0.0,0.000,0.000,0.000,0.3
8,2018-01-01,0,광진구,33.0,16.0,0.025,0.004,0.018,0.4
9,2018-01-01,0,동대문구,35.0,13.0,0.022,0.005,0.015,0.5


In [41]:


# 중복된 행을 그룹화하고 각 그룹의 컬럼값을 합침
grouped = air_df.groupby(['date', 'time', 'gu'])[['PM10', 'PM2.5', 'O3', 'SO2', 'NO2', 'CO']].sum().reset_index()

# 중복된 행을 제거
# 중복된 행을 제거
air_df = air_df.drop_duplicates(subset=['date', 'time', 'gu'])

# 합쳐진 값을 원래 데이터프레임에 추가
air_df = pd.merge(air_df, grouped, on=['date', 'time', 'gu'], suffixes=('', '_sum'))

# 필요없는 중복된 컬럼 제거
air_df = air_df.drop(columns=['PM10_sum', 'PM2.5_sum', 'O3_sum', 'SO2_sum', 'NO2_sum', 'CO_sum'])
# 결과 출력
print(air_df)


              date  time    gu  PM10  PM2.5     O3    SO2    NO2   CO
0       2018-01-01     0   종로구  29.0   17.0  0.022  0.006  0.021  0.4
1       2018-01-01     0    중구  34.0   19.0  0.020  0.004  0.020  0.5
2       2018-01-01     0   용산구  29.0   14.0  0.013  0.004  0.023  0.3
3       2018-01-01     0   은평구  43.0   25.0  0.017  0.003  0.018  0.6
4       2018-01-01     0  서대문구   0.0    0.0  0.000  0.000  0.000  0.0
...            ...   ...   ...   ...    ...    ...    ...    ...  ...
218995  2018-12-31    23   관악구  36.0   24.0  0.004  0.005  0.059  0.6
218996  2018-12-31    23   서초구  36.0   13.0  0.002  0.004  0.052  0.5
218997  2018-12-31    23   강남구  35.0   26.0  0.002  0.005  0.044  0.7
218998  2018-12-31    23   송파구  43.0   27.0  0.002  0.004  0.057  0.9
218999  2018-12-31    23   강동구  53.0   34.0  0.002  0.006  0.072  1.1

[219000 rows x 9 columns]


In [44]:
air_df.nunique()


date     365
time      24
gu        25
PM10     165
PM2.5    119
O3       106
SO2       65
NO2      106
CO        52
dtype: int64

In [45]:
gu_df['gu'].unique()


array(['서대문구', '종로구', '성북구', '중구', '용산구', '도봉구', '노원구', '광진구', '중랑구',
       '강동구', '서초구', '송파구', '강남구', '금천구', '구로구', '강서구', '양천구', '은평구',
       '마포구', '영등포구', '성동구', '동작구', '관악구', '동대문구'], dtype=object)

In [46]:
gu_df.to_csv('new_gu.csv',encoding='utf-8')

In [47]:
air_df['gu'].unique()

array(['종로구', '중구', '용산구', '은평구', '서대문구', '마포구', '성동구', '광진구', '동대문구',
       '중랑구', '성북구', '강북구', '도봉구', '노원구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'], dtype=object)

In [48]:
import pandas as pd

# gu_df와 air_df의 'gu' 열에서 고유한 값 얻기
unique_gu_df = gu_df['gu'].unique()
unique_air_df = air_df['gu'].unique()

# 겹치지 않는 값을 찾기
not_in_both = set(unique_gu_df) ^ set(unique_air_df)

# 결과 출력
print("gu_df에만 있는 값:", set(unique_gu_df) - set(unique_air_df))
print("air_df에만 있는 값:", set(unique_air_df) - set(unique_gu_df))
print("둘 다에 있는 값:", set(unique_gu_df) & set(unique_air_df))
print("둘 중 하나에만 있는 값:", not_in_both)


gu_df에만 있는 값: set()
air_df에만 있는 값: {'강북구'}
둘 다에 있는 값: {'용산구', '금천구', '서대문구', '은평구', '관악구', '마포구', '중랑구', '중구', '구로구', '도봉구', '서초구', '성동구', '노원구', '성북구', '종로구', '강동구', '강서구', '광진구', '강남구', '동작구', '영등포구', '동대문구', '송파구', '양천구'}
둘 중 하나에만 있는 값: {'강북구'}


In [56]:
# 'gu' 열이 '강북구'인 값을 제외한 새로운 데이터프레임 생성
new_air_df = air_df[air_df['gu'] != '강북구']


In [57]:
new_air_df.nunique()

date     365
time      24
gu        24
PM10     165
PM2.5    119
O3       106
SO2       65
NO2      106
CO        52
dtype: int64

Missing values per column:
 date     0
time     0
gu       0
PM10     0
PM2.5    0
O3       0
SO2      0
NO2      0
CO       0
dtype: int64


In [50]:
duplicated_rows = air_df.duplicated()
print("Number of duplicated rows:", duplicated_rows.sum())


Number of duplicated rows: 0


In [51]:
duplicate_rows = air_df[air_df.duplicated()]
print("Duplicate rows:\n", duplicate_rows)


Duplicate rows:
 Empty DataFrame
Columns: [date, time, gu, PM10, PM2.5, O3, SO2, NO2, CO]
Index: []
